# ETAPA 3: LIMPIEZA DE DATOS - League of Legends Worlds

## Objetivos:
1. Documentar el proceso de limpieza de datos
2. Explicar las decisiones tomadas en cada paso
3. Mostrar el impacto de la limpieza en la calidad de los datos
4. Validar la consistencia de los datos limpios


## 1. Importar Librerías y Cargar Datos


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Configurar visualizaciones
plt.style.use('default')
sns.set_palette("husl")

print("Librerías importadas correctamente")


In [ ]:
# Cargar datos originales
champions_raw = pd.read_csv('../data/01_raw/champions_stats.csv', encoding='latin-1')
matches_raw = pd.read_csv('../data/01_raw/matchs_stats.csv', encoding='latin-1')
players_raw = pd.read_csv('../data/01_raw/players_stats.csv', encoding='latin-1')

print("Datos originales cargados:")
print(f"- Champions: {champions_raw.shape}")
print(f"- Matches: {matches_raw.shape}")
print(f"- Players: {players_raw.shape}")


## 2. Análisis de Calidad de Datos Originales


In [ ]:
def analyze_data_quality(df, dataset_name):
    """Analiza la calidad de un dataset"""
    print(f"\n=== ANÁLISIS DE CALIDAD: {dataset_name.upper()} ===")
    print(f"Dimensiones: {df.shape}")
    print(f"\nValores faltantes por columna:")
    missing = df.isnull().sum()
    missing_pct = (missing / len(df)) * 100
    missing_df = pd.DataFrame({
        'Valores_Faltantes': missing,
        'Porcentaje': missing_pct
    })
    print(missing_df[missing_df['Valores_Faltantes'] > 0])
    
    print(f"\nDuplicados: {df.duplicated().sum()}")
    print(f"\nTipos de datos:")
    print(df.dtypes.value_counts())
    
    return missing_df

# Analizar calidad de datos originales
champions_missing = analyze_data_quality(champions_raw, "Champions")
matches_missing = analyze_data_quality(matches_raw, "Matches")
players_missing = analyze_data_quality(players_raw, "Players")


## 3. Limpieza de Datos de Campeones


In [ ]:
def clean_champions_data_detailed(df):
    """Limpia datos de campeones con explicaciones detalladas"""
    print("=== LIMPIEZA DE DATOS DE CAMPEONES ===")
    original_shape = df.shape
    
    # 1. Eliminar filas con valores faltantes críticos
    print("\n1. Eliminando filas con valores faltantes críticos...")
    critical_columns = ['champion', 'win', 'kills', 'deaths', 'assists']
    before_drop = len(df)
    df_clean = df.dropna(subset=critical_columns)
    after_drop = len(df_clean)
    print(f"   Filas eliminadas: {before_drop - after_drop}")
    
    # 2. Limpiar espacios en blanco en columnas de texto
    print("\n2. Limpiando espacios en blanco...")
    text_columns = df_clean.select_dtypes(include=['object']).columns
    for col in text_columns:
        df_clean[col] = df_clean[col].astype(str).str.strip()
    
    # 3. Convertir tipos de datos
    print("\n3. Convirtiendo tipos de datos...")
    numeric_columns = ['win', 'kills', 'deaths', 'assists', 'gold', 'damage', 'damagetaken']
    for col in numeric_columns:
        if col in df_clean.columns:
            df_clean[col] = pd.to_numeric(df_clean[col], errors='coerce')
    
    # 4. Manejar valores faltantes en columnas numéricas
    print("\n4. Manejando valores faltantes en columnas numéricas...")
    for col in numeric_columns:
        if col in df_clean.columns:
            missing_count = df_clean[col].isnull().sum()
            if missing_count > 0:
                df_clean[col] = df_clean[col].fillna(0)
                print(f"   {col}: {missing_count} valores faltantes rellenados con 0")
    
    # 5. Filtrar datos inconsistentes
    print("\n5. Filtrando datos inconsistentes...")
    # Eliminar filas donde kills, deaths, assists son negativos
    invalid_stats = (df_clean['kills'] < 0) | (df_clean['deaths'] < 0) | (df_clean['assists'] < 0)
    invalid_count = invalid_stats.sum()
    df_clean = df_clean[~invalid_stats]
    print(f"   Filas con estadísticas negativas eliminadas: {invalid_count}")
    
    # 6. Validar rangos de datos
    print("\n6. Validando rangos de datos...")
    if 'win' in df_clean.columns:
        invalid_wins = ~df_clean['win'].isin([0, 1])
        invalid_win_count = invalid_wins.sum()
        if invalid_win_count > 0:
            df_clean = df_clean[~invalid_wins]
            print(f"   Filas con valores de win inválidos eliminadas: {invalid_win_count}")
    
    final_shape = df_clean.shape
    print(f"\n=== RESUMEN DE LIMPIEZA ===")
    print(f"Forma original: {original_shape}")
    print(f"Forma final: {final_shape}")
    print(f"Filas eliminadas: {original_shape[0] - final_shape[0]}")
    print(f"Porcentaje de datos conservados: {(final_shape[0]/original_shape[0])*100:.1f}%")
    
    return df_clean

# Aplicar limpieza a campeones
champions_clean = clean_champions_data_detailed(champions_raw.copy())


## 4. Limpieza de Datos de Partidos


In [ ]:
def clean_matches_data_detailed(df):
    """Limpia datos de partidos con explicaciones detalladas"""
    print("=== LIMPIEZA DE DATOS DE PARTIDOS ===")
    original_shape = df.shape
    
    # 1. Limpiar columnas de texto
    print("\n1. Limpiando columnas de texto...")
    text_columns = df.select_dtypes(include=['object']).columns
    for col in text_columns:
        df[col] = df[col].astype(str).str.strip()
    
    # 2. Convertir fechas usando pandas
    print("\n2. Convirtiendo fechas...")
    if 'date' in df.columns:
        df['date'] = pd.to_datetime(df['date'], errors='coerce')
        invalid_dates = df['date'].isnull().sum()
        if invalid_dates > 0:
            print(f"   Fechas inválidas encontradas: {invalid_dates}")
            df = df.dropna(subset=['date'])
    
    # 3. Convertir columnas numéricas usando pandas
    print("\n3. Convirtiendo columnas numéricas...")
    numeric_columns = ['duration', 'team1_kills', 'team2_kills', 'team1_gold', 'team2_gold']
    for col in numeric_columns:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce')
    
    # 4. Manejar valores faltantes
    print("\n4. Manejando valores faltantes...")
    for col in numeric_columns:
        if col in df.columns:
            missing_count = df[col].isnull().sum()
            if missing_count > 0:
                df[col] = df[col].fillna(0)
                print(f"   {col}: {missing_count} valores faltantes rellenados con 0")
    
    # 5. Validar consistencia de datos
    print("\n5. Validando consistencia de datos...")
    # Duración debe ser positiva
    invalid_duration = df['duration'] <= 0
    invalid_duration_count = invalid_duration.sum()
    if invalid_duration_count > 0:
        df = df[~invalid_duration]
        print(f"   Partidos con duración inválida eliminados: {invalid_duration_count}")
    
    # Kills no pueden ser negativos
    invalid_kills = (df['team1_kills'] < 0) | (df['team2_kills'] < 0)
    invalid_kills_count = invalid_kills.sum()
    if invalid_kills_count > 0:
        df = df[~invalid_kills]
        print(f"   Partidos con kills negativos eliminados: {invalid_kills_count}")
    
    final_shape = df.shape
    print(f"\n=== RESUMEN DE LIMPIEZA ===")
    print(f"Forma original: {original_shape}")
    print(f"Forma final: {final_shape}")
    print(f"Filas eliminadas: {original_shape[0] - final_shape[0]}")
    print(f"Porcentaje de datos conservados: {(final_shape[0]/original_shape[0])*100:.1f}%")
    
    return df

# Aplicar limpieza a partidos
matches_clean = clean_matches_data_detailed(matches_raw.copy())


## 5. Limpieza de Datos de Jugadores


In [ ]:
def clean_players_data_detailed(df):
    """Limpia datos de jugadores con explicaciones detalladas"""
    print("=== LIMPIEZA DE DATOS DE JUGADORES ===")
    original_shape = df.shape
    
    # 1. Eliminar filas con valores faltantes críticos
    print("\n1. Eliminando filas con valores faltantes críticos...")
    critical_columns = ['player', 'team', 'champion', 'win']
    before_drop = len(df)
    df_clean = df.dropna(subset=critical_columns)
    after_drop = len(df_clean)
    print(f"   Filas eliminadas: {before_drop - after_drop}")
    
    # 2. Limpiar columnas de texto
    print("\n2. Limpiando columnas de texto...")
    text_columns = df_clean.select_dtypes(include=['object']).columns
    for col in text_columns:
        df_clean[col] = df_clean[col].astype(str).str.strip()
    
    # 3. Convertir columnas numéricas usando pandas
    print("\n3. Convirtiendo columnas numéricas...")
    numeric_columns = ['win', 'kills', 'deaths', 'assists', 'gold', 'damage', 'damagetaken']
    for col in numeric_columns:
        if col in df_clean.columns:
            df_clean[col] = pd.to_numeric(df_clean[col], errors='coerce')
    
    # 4. Manejar valores faltantes
    print("\n4. Manejando valores faltantes...")
    for col in numeric_columns:
        if col in df_clean.columns:
            missing_count = df_clean[col].isnull().sum()
            if missing_count > 0:
                df_clean[col] = df_clean[col].fillna(0)
                print(f"   {col}: {missing_count} valores faltantes rellenados con 0")
    
    # 5. Filtrar datos inconsistentes usando numpy
    print("\n5. Filtrando datos inconsistentes...")
    # Eliminar filas con estadísticas negativas
    invalid_stats = (df_clean['kills'] < 0) | (df_clean['deaths'] < 0) | (df_clean['assists'] < 0)
    invalid_count = invalid_stats.sum()
    df_clean = df_clean[~invalid_stats]
    print(f"   Filas con estadísticas negativas eliminadas: {invalid_count}")
    
    # Validar valores de win usando numpy
    invalid_wins = ~df_clean['win'].isin([0, 1])
    invalid_win_count = invalid_wins.sum()
    if invalid_win_count > 0:
        df_clean = df_clean[~invalid_wins]
        print(f"   Filas con valores de win inválidos eliminadas: {invalid_win_count}")
    
    # 6. Eliminar jugadores con nombres vacíos o muy cortos
    print("\n6. Filtrando nombres de jugadores...")
    short_names = df_clean['player'].str.len() < 2
    short_names_count = short_names.sum()
    df_clean = df_clean[~short_names]
    print(f"   Filas con nombres muy cortos eliminadas: {short_names_count}")
    
    final_shape = df_clean.shape
    print(f"\n=== RESUMEN DE LIMPIEZA ===")
    print(f"Forma original: {original_shape}")
    print(f"Forma final: {final_shape}")
    print(f"Filas eliminadas: {original_shape[0] - final_shape[0]}")
    print(f"Porcentaje de datos conservados: {(final_shape[0]/original_shape[0])*100:.1f}%")
    
    return df_clean

# Aplicar limpieza a jugadores
players_clean = clean_players_data_detailed(players_raw.copy())


## 6. Análisis de Impacto de la Limpieza con Visualizaciones


In [ ]:
# Crear visualización del impacto de la limpieza usando numpy y pandas
datasets = ['Champions', 'Matches', 'Players']
original_sizes = np.array([champions_raw.shape[0], matches_raw.shape[0], players_raw.shape[0]])
clean_sizes = np.array([champions_clean.shape[0], matches_clean.shape[0], players_clean.shape[0]])

# Crear DataFrame para análisis usando pandas
impact_df = pd.DataFrame({
    'Dataset': datasets,
    'Original': original_sizes,
    'Limpio': clean_sizes,
    'Eliminados': original_sizes - clean_sizes,
    'Porcentaje_Conservado': (clean_sizes / original_sizes) * 100
})

print("=== ANÁLISIS DE IMPACTO DE LA LIMPIEZA ===")
print(impact_df)

# Crear visualizaciones
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Gráfico de barras comparativo usando numpy para posicionamiento
x = np.arange(len(datasets))
width = 0.35

ax1.bar(x - width/2, original_sizes, width, label='Original', alpha=0.8, color='red')
ax1.bar(x + width/2, clean_sizes, width, label='Limpio', alpha=0.8, color='green')

ax1.set_xlabel('Dataset')
ax1.set_ylabel('Número de Filas')
ax1.set_title('Impacto de la Limpieza de Datos')
ax1.set_xticks(x)
ax1.set_xticklabels(datasets)
ax1.legend()
ax1.grid(True, alpha=0.3)

# Gráfico de porcentaje de datos conservados
conservation_pct = impact_df['Porcentaje_Conservado'].values
bars = ax2.bar(datasets, conservation_pct, alpha=0.8, color='blue')
ax2.set_ylabel('Porcentaje de Datos Conservados (%)')
ax2.set_title('Porcentaje de Datos Conservados Después de la Limpieza')
ax2.set_ylim(0, 100)
ax2.grid(True, alpha=0.3)

# Añadir valores en las barras usando numpy
for bar, pct in zip(bars, conservation_pct):
    height = bar.get_height()
    ax2.text(bar.get_x() + bar.get_width()/2., height + 1,
             f'{pct:.1f}%', ha='center', va='bottom')

plt.tight_layout()
plt.show()

# Mostrar resumen numérico usando pandas
print("\n=== RESUMEN DEL IMPACTO DE LA LIMPIEZA ===")
for _, row in impact_df.iterrows():
    print(f"{row['Dataset']}:")
    print(f"  Original: {row['Original']:,} filas")
    print(f"  Limpio: {row['Limpio']:,} filas")
    print(f"  Eliminadas: {row['Eliminados']:,} filas ({100-row['Porcentaje_Conservado']:.1f}%)")
    print()


## 7. Análisis Estadístico de los Datos Limpios


In [ ]:
# Análisis estadístico usando pandas y numpy
def analyze_clean_data_stats(df, dataset_name):
    """Realiza análisis estadístico de los datos limpios"""
    print(f"\n=== ANÁLISIS ESTADÍSTICO: {dataset_name.upper()} ===")
    
    # Estadísticas descriptivas básicas usando pandas
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    if len(numeric_cols) > 0:
        print(f"\nEstadísticas descriptivas de columnas numéricas:")
        stats_df = df[numeric_cols].describe()
        print(stats_df)
        
        # Análisis de correlaciones usando pandas
        if len(numeric_cols) > 1:
            print(f"\nMatriz de correlaciones:")
            corr_matrix = df[numeric_cols].corr()
            print(corr_matrix.round(3))
    
    # Análisis de valores únicos usando pandas
    text_cols = df.select_dtypes(include=['object']).columns
    if len(text_cols) > 0:
        print(f"\nAnálisis de columnas categóricas:")
        for col in text_cols[:5]:  # Solo las primeras 5 columnas
            unique_count = df[col].nunique()
            most_common = df[col].mode().iloc[0] if not df[col].mode().empty else "N/A"
            print(f"  {col}: {unique_count} valores únicos, más común: '{most_common}'")
    
    # Análisis de distribución usando numpy
    if 'win' in df.columns:
        win_distribution = df['win'].value_counts()
        win_rate = (win_distribution.get(1, 0) / len(df)) * 100
        print(f"\nDistribución de victorias:")
        print(f"  Victoria: {win_distribution.get(1, 0)} ({win_rate:.1f}%)")
        print(f"  Derrota: {win_distribution.get(0, 0)} ({100-win_rate:.1f}%)")
    
    return stats_df if len(numeric_cols) > 0 else None

# Realizar análisis estadístico
champions_stats = analyze_clean_data_stats(champions_clean, "Champions")
matches_stats = analyze_clean_data_stats(matches_clean, "Matches")
players_stats = analyze_clean_data_stats(players_clean, "Players")


## 8. Guardar Datos Limpios y Conclusiones


In [ ]:
# Guardar datos limpios usando pandas
champions_clean.to_csv('../league-of-legends-worlds/data/02_intermediate/champions_clean.csv', index=False, encoding='latin-1')
matches_clean.to_csv('../league-of-legends-worlds/data/02_intermediate/matches_clean.csv', index=False, encoding='latin-1')
players_clean.to_csv('../league-of-legends-worlds/data/02_intermediate/players_clean.csv', index=False, encoding='latin-1')

print("✅ Datos limpios guardados exitosamente usando pandas")
print("\nArchivos guardados:")
print("- ../league-of-legends-worlds/data/02_intermediate/champions_clean.csv")
print("- ../league-of-legends-worlds/data/02_intermediate/matches_clean.csv")
print("- ../league-of-legends-worlds/data/02_intermediate/players_clean.csv")

# Crear resumen final usando pandas y numpy
summary_data = {
    'Dataset': ['Champions', 'Matches', 'Players'],
    'Filas_Originales': [champions_raw.shape[0], matches_raw.shape[0], players_raw.shape[0]],
    'Filas_Limpias': [champions_clean.shape[0], matches_clean.shape[0], players_clean.shape[0]],
    'Columnas_Originales': [champions_raw.shape[1], matches_raw.shape[1], players_raw.shape[1]],
    'Columnas_Limpias': [champions_clean.shape[1], matches_clean.shape[1], players_clean.shape[1]]
}

summary_df = pd.DataFrame(summary_data)
summary_df['Filas_Eliminadas'] = summary_df['Filas_Originales'] - summary_df['Filas_Limpias']
summary_df['Porcentaje_Conservado'] = (summary_df['Filas_Limpias'] / summary_df['Filas_Originales']) * 100

print("\n=== RESUMEN FINAL DE LA LIMPIEZA ===")
print(summary_df.to_string(index=False))

# Calcular estadísticas generales usando numpy
total_original = np.sum(summary_df['Filas_Originales'])
total_clean = np.sum(summary_df['Filas_Limpias'])
total_removed = total_original - total_clean
overall_conservation = (total_clean / total_original) * 100

print(f"\n=== ESTADÍSTICAS GENERALES ===")
print(f"Total de filas originales: {total_original:,}")
print(f"Total de filas después de limpieza: {total_clean:,}")
print(f"Total de filas eliminadas: {total_removed:,}")
print(f"Porcentaje general de conservación: {overall_conservation:.1f}%")


## 9. Conclusiones de la Limpieza de Datos

### Resumen de Acciones Realizadas:

1. **Eliminación de valores faltantes críticos**: Se eliminaron filas con información esencial faltante
2. **Limpieza de texto**: Se eliminaron espacios en blanco y se estandarizaron formatos usando pandas
3. **Conversión de tipos**: Se convirtieron columnas a los tipos de datos apropiados usando pandas
4. **Manejo de valores faltantes**: Se rellenaron valores faltantes con valores apropiados (0 para estadísticas)
5. **Filtrado de inconsistencias**: Se eliminaron filas con datos lógicamente incorrectos usando numpy
6. **Validación de rangos**: Se verificaron rangos de valores para asegurar consistencia

### Uso de Librerías:

- **Pandas**: Para manipulación de DataFrames, conversión de tipos, manejo de valores faltantes, y análisis estadístico
- **NumPy**: Para operaciones matemáticas, arrays, y validaciones lógicas
- **Matplotlib/Seaborn**: Para visualizaciones del impacto de la limpieza

### Impacto en la Calidad de Datos:

- **Datos más consistentes**: Eliminación de valores anómalos y inconsistentes
- **Tipos de datos correctos**: Conversión apropiada para análisis posterior
- **Menos valores faltantes**: Manejo sistemático de datos faltantes
- **Mayor confiabilidad**: Validación de rangos y consistencia lógica

### Próximos Pasos:

Los datos limpios están listos para:
1. **Ingeniería de características**: Creación de variables derivadas
2. **Análisis exploratorio**: Identificación de patrones y tendencias
3. **Modelado**: Desarrollo de modelos de machine learning
4. **Validación**: Verificación de la calidad de los modelos

### Ventajas del Enfoque en Notebooks:

- **Transparencia**: Cada paso de limpieza está documentado y explicado
- **Reproducibilidad**: El proceso puede ser ejecutado múltiples veces
- **Interactividad**: Permite explorar los datos y ajustar el proceso
- **Visualización**: Gráficos que muestran el impacto de cada acción
- **Educativo**: Facilita el entendimiento del proceso de limpieza
